# Part 7 - Federated Learning with FederatedDataset

Here we introduce a new tool for using federated datasets. We have created a `FederatedDataset` class which is intended to be used like the PyTorch Dataset class, and is given to a federated data loader `FederatedDataLoader` which will iterate on it in a federated fashion.


Authors:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Théo Ryffel - GitHub: [@LaRiffle](https://github.com/LaRiffle)

We use the sandbox that we discovered last lesson

In [1]:
import torch as th
import syft as sy
sy.create_sandbox(globals(), verbose=False)

Setting up Sandbox...
Done!


Then search for a dataset

In [2]:
boston_data = grid.search("#boston", "#data", verbose=False, return_counter=False)
boston_target = grid.search("#boston", "#target", verbose=False, return_counter=False)

We load a model and an optimizer

In [3]:
n_features = boston_data['alice'][0].shape[1]
n_targets = 1
model = th.nn.Linear(n_features, n_targets)
optimizer = th.optim.SGD(params=model.parameters(),lr=0.0000001)

Here we cast the data fetched in a `FederatedDataset`. See the workers which hold part of the data.

In [4]:
# Cast the result in BaseDatasets
datasets = []
for worker in boston_data.keys():
    dataset = sy.BaseDataset(boston_data[worker][0], boston_target[worker][0])
    datasets.append(dataset)

# Build the FederatedDataset object
dataset = sy.FederatedDataset(datasets)
print(dataset.workers)

['bob', 'theo', 'jason', 'alice', 'andy', 'jon']


We put it in a `FederatedDataLoader` and specify options

In [5]:
train_loader = sy.FederatedDataLoader(dataset, batch_size=4, shuffle=False, drop_last=False)

And finally we iterate over epochs. You can see how similar this is compared to pure and local PyTorch training!

In [6]:
epochs = 10
for epoch in range(1, epochs + 1):
    loss_accum = 0
    for batch_idx, (data, target) in enumerate(train_loader):

        model.send(data.location)
        
        optimizer.zero_grad()
        pred = model(data)
        loss = ((pred - target)**2).sum()
        loss.backward()
        optimizer.step()
        
        model.get()
        loss = loss.get()
        
        loss_accum += float(loss)
        
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * data.shape[0], len(train_loader),
                       100. * batch_idx / len(train_loader), loss.item()))
            
            
    print('Total loss', loss_accum)


Train Epoch: 1 [0/127 (0%)]	Loss: 52030.949219
Train Epoch: 1 [80/127 (16%)]	Loss: 4476.936035
Train Epoch: 1 [160/127 (31%)]	Loss: 3504.968750
Train Epoch: 1 [240/127 (47%)]	Loss: 1739.490845
Train Epoch: 1 [320/127 (63%)]	Loss: 72.340225
Train Epoch: 1 [400/127 (79%)]	Loss: 148.348114
Train Epoch: 1 [480/127 (94%)]	Loss: 10308.703125
Total loss 428401.8585510254
Train Epoch: 2 [0/127 (0%)]	Loss: 26.898750
Train Epoch: 2 [40/127 (16%)]	Loss: 35.393768
Train Epoch: 2 [80/127 (31%)]	Loss: 2151.689453
Train Epoch: 2 [120/127 (47%)]	Loss: 1237.475952
Train Epoch: 2 [160/127 (63%)]	Loss: 23.571852
Train Epoch: 2 [200/127 (79%)]	Loss: 182.861404
Train Epoch: 2 [240/127 (94%)]	Loss: 109.622910
Total loss 40875.63382220268
Train Epoch: 3 [0/127 (0%)]	Loss: 67.843719
Train Epoch: 3 [40/127 (16%)]	Loss: 42.780453
Train Epoch: 3 [80/127 (31%)]	Loss: 2177.287354
Train Epoch: 3 [120/127 (47%)]	Loss: 1207.351318
Train Epoch: 3 [160/127 (63%)]	Loss: 22.607536
Train Epoch: 3 [200/127 (79%)]	Loss: 179

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)